# Activity: Explore hypothesis testing

## Introduction

You work for an environmental think tank called Repair Our Air (ROA). ROA is formulating policy recommendations to improve the air quality in America, using the Environmental Protection Agency's Air Quality Index (AQI) to guide their decision making. An AQI value close to 0 signals "little to no" public health concern, while higher values are associated with increased risk to public health. 

They've tasked you with leveraging AQI data to help them prioritize their strategy for improving air quality in America.

ROA is considering the following decisions. For each, construct a hypothesis test and an accompanying visualization, using your results of that test to make a recommendation:

1. ROA is considering a metropolitan-focused approach. Within California, they want to know if the mean AQI in Los Angeles County is statistically different from the rest of California.
2. With limited resources, ROA has to choose between New York and Ohio for their next regional office. Does New York have a lower AQI than Ohio?
3. A new policy will affect those states with a mean AQI of 10 or greater. Can you rule out Michigan from being affected by this new policy?

**Notes:**
1. For your analysis, you'll default to a 5% level of significance.
2. Throughout the lab, for two-sample t-tests, use Welch's t-test (i.e., setting the `equal_var` parameter to `False` in `scipy.stats.ttest_ind()`). This will account for the possibly unequal variances between the two groups in the comparison.

## Step 1: Imports

To proceed with your analysis, import `pandas` and `numpy`. To conduct your hypothesis testing, import `stats` from `scipy`.

#### Import Packages

In [12]:
# Import relevant packages

### YOUR CODE HERE ###
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns

You are also provided with a dataset with national Air Quality Index (AQI) measurements by state over time for this analysis. `Pandas` was used to import the file `c4_epa_air_quality.csv` as a dataframe named `aqi`. As shown in this cell, the dataset has been automatically loaded in for you. You do not need to download the .csv file, or provide more code, in order to access the dataset and proceed with this lab. Please continue with this activity by completing the following instructions.

**Note:** For purposes of your analysis, you can assume this data is randomly sampled from a larger population.

#### Load Dataset

In [13]:
# RUN THIS CELL TO IMPORT YOUR DATA.

### YOUR CODE HERE ###
aqi = pd.read_csv('c4_epa_air_quality.csv')

## Step 2: Data Exploration

### Before proceeding to your deliverables, explore your datasets.

Use the following space to surface descriptive statistics about your data. In particular, explore whether you believe the research questions you were given are readily answerable with this data.

In [14]:
# Explore your dataframe `aqi` here:

### YOUR CODE HERE ###
# Remove null values in dataframe
aqi.dropna()

# Look at dataframe
aqi.head()

,Unnamed: 0,date_local,state_name,county_name,city_name,local_site_name,parameter_name,units_of_measure,arithmetic_mean,aqi
0,0,2018-01-01,Arizona,Maricopa,Buckeye,BUCKEYE,Carbon monoxide,Parts per million,0.473684,7
1,1,2018-01-01,Ohio,Belmont,Shadyside,Shadyside,Carbon monoxide,Parts per million,0.263158,5
2,2,2018-01-01,Wyoming,Teton,Not in a city,Yellowstone National Park - Old Faithful Snow ...,Carbon monoxide,Parts per million,0.111111,2
3,3,2018-01-01,Pennsylvania,Philadelphia,Philadelphia,North East Waste (NEW),Carbon monoxide,Parts per million,0.300000,3
4,4,2018-01-01,Iowa,Polk,Des Moines,CARPENTER,Carbon monoxide,Parts per million,0.215789,3


In [50]:
# Show important statistics per state sorted by mean aqi
print("Descriptive statistics for aqi levels in each state")
aqi.groupby(['state_name']).aqi.describe().sort_values(by='count',ascending=False)

Descriptive statistics for aqi levels in each state


,count,mean,std,min,25%,50%,75%,max
state_name,,,,,,,,
California,66.0,12.121212,7.301244,1.0,7.00,11.0,16.00,40.0
Arizona,14.0,15.214286,14.104983,2.0,5.50,10.0,19.75,50.0
Ohio,12.0,3.333333,1.302678,2.0,2.75,3.0,3.50,6.0
Florida,12.0,5.500000,2.430862,1.0,4.50,6.0,7.00,9.0
New York,10.0,2.500000,0.527046,2.0,2.00,2.5,3.00,3.0
Texas,10.0,2.700000,0.948683,2.0,2.00,2.5,3.00,5.0
Pennsylvania,10.0,2.900000,1.663330,0.0,2.00,3.0,3.00,6.0
Colorado,9.0,5.000000,1.322876,3.0,5.00,5.0,6.00,7.0
Michigan,9.0,8.111111,3.257470,2.0,7.00,8.0,10.00,13.0


In [32]:
# Dataframe shape
print("Dataframe shape:")
aqi.shape

Dataframe shape:


(260, 10)

<details>
  <summary><h4><strong>HINT 1</strong></h4></summary>

  Consider referring to the material on descriptive statisics.
</details>

<details>
  <summary><h4><strong>HINT 2</strong></h4></summary>

  Consider using `pandas` or `numpy` to explore the `aqi` dataframe.
</details>

<details>
  <summary><h4><strong>HINT 3</strong></h4></summary>

Any of the following functions may be useful:
- `pandas`: `describe()`,`value_counts()`,`shape()`, `head()`
- `numpy`: `unique()`,`mean()`
    
</details>

#### **Question 1: From the preceding data exploration, what do you recognize?**

We notice that eventhough california is the state with most entries it is not the state with the highest mean aqi. We also see that Arizon is both second in the frequecy count and mean aqi level, this could be indication that Arizona's true aqi might be at some concerning level.

## Step 3. Statistical Tests

Before you proceed, recall the following steps for conducting hypothesis testing:

1. Formulate the null hypothesis and the alternative hypothesis.<br>
2. Set the significance level.<br>
3. Determine the appropriate test procedure.<br>
4. Compute the p-value.<br>
5. Draw your conclusion.

### Hypothesis 1: ROA is considering a metropolitan-focused approach. Within California, they want to know if the mean AQI in Los Angeles County is statistically different from the rest of California.

Before proceeding with your analysis, it will be helpful to subset the data for your comparison.

In [60]:
# Create dataframes for each sample being compared in your test

### YOUR CODE HERE ###
California_aqi = aqi[aqi.state_name == 'California']
LA_aqi = aqi[aqi.county_name == 'Los Angeles']

print("California dataframe shape:",California_aqi.shape)
print("Los Angeles dataframe shape:",LA_aqi.shape)

California dataframe shape: (66, 10)
Los Angeles dataframe shape: (14, 10)


<details>
  <summary><h4><strong>HINT 1</strong></h4></summary>

  Consider referencing the material on subsetting dataframes.  
</details>

<details>
  <summary><h4><strong>HINT 2</strong></h4></summary>

  Consider creating two dataframes, one for Los Angeles, and one for all other California observations.
</details>

<details>
  <summary><h4><strong>HINT 3</strong></h4></summary>

For your first dataframe, filter to `county_name` of `Los Angeles`. For your second dataframe, filter to `state_name` of `Calfornia` and `county_name` not equal to `Los Angeles`.
    
</details>

#### Formulate your hypothesis:

**Formulate your null and alternative hypotheses:**

*   $H_0$: There is no difference in the mean AQI between Los Angeles County and the rest of California.
*   $H_A$: There is a difference in the mean AQI between Los Angeles County and the rest of California.


#### Set the significance level:

In [47]:
# For this analysis, the significance level is 5%

### YOUR CODE HERE
SL = 0.05

#### Determine the appropriate test procedure:

Here, you are comparing the sample means between two independent samples. Therefore, you will utilize a **two-sample  𝑡-test**.

#### Compute the P-value

In [79]:
# Compute your p-value here

### YOUR CODE HERE ###
stats.ttest_ind(a=LA_aqi.aqi,b=California_aqi.aqi,equal_var=False) 

Ttest_indResult(statistic=1.664076657643435, pvalue=0.11435587227090667)

<details>
  <summary><h4><strong>HINT 1</strong></h4></summary>

  Consider referencing the material on how to perform a two-sample t-test.
</details>

<details>
  <summary><h4><strong>HINT 2</strong></h4></summary>

  In `ttest_ind()`, a is the aqi column from our "Los Angeles" dataframe, and b is the aqi column from the "Other California" dataframe.
</details>

<details>
  <summary><h4><strong>HINT 3</strong></h4></summary>

  Be sure to set `equal_var` = False.

</details>

#### **Question 2. What is your P-value for hypothesis 1, and what does this indicate for your null hypothesis?**

The P-value for hypothesis 1 is 0.114 or 11.4% which is higher than 0.05 or 5%. Since our P-value is higher than our significance level we fail to reject the null hypothesis, therefore, there is no statistical significance between the difference of the mean AQI of California and LA.

### Hypothesis 2: With limited resources, ROA has to choose between New York and Ohio for their next regional office. Does New York have a lower AQI than Ohio?

Before proceeding with your analysis, it will be helpful to subset the data for your comparison.

In [75]:
# Create dataframes for each sample being compared in your test

### YOUR CODE HERE ###
NY_df = aqi[aqi.state_name=='New York']
Ohio_df = aqi[aqi.state_name=='Ohio']

print("NY dataframe shape:",NY_df.shape)
print("Ohio dataframe shape:",Ohio_df.shape,'\n')

print('Mean AQI')
print("New York:",NY_df.aqi.mean())
print("Ohio:",Ohio_df.aqi.mean())

NY dataframe shape: (10, 10)
Ohio dataframe shape: (12, 10) 

Mean AQI
New York: 2.5
Ohio: 3.3333333333333335


<details>
  <summary><h4><strong>HINT 1</strong></h4></summary>

  Consider referencing the materials on subsetting dataframes.  
</details>

<details>
  <summary><h4><strong>HINT 2</strong></h4></summary>

  Consider creating two dataframes, one for New York, and one for Ohio observations.
</details>

<details>
  <summary><h4><strong>HINT 3</strong></h4></summary>

For your first dataframe, filter to `state_name` of `New York`. For your second dataframe, filter to `state_name` of `Ohio`.
    
</details>

#### Formulate your hypothesis:

**Formulate your null and alternative hypotheses:**

*   $H_0$: The mean AQI of New York is greater than or equal to that of Ohio.
*   $H_A$: The mean AQI of New York is **below** that of Ohio.


#### Significance Level (remains at 5%)

#### Determine the appropriate test procedure:

Here, you are comparing the sample means between two independent samples in one direction. Therefore, you will utilize a **two-sample  𝑡-test**.

#### Compute the P-value

In [80]:
# Computer your p-value here

### YOUR CODE HERE ###
tstat, pvalue = stats.ttest_ind(a=NY_df.aqi,b=Ohio_df.aqi,equal_var=False,alternative='less')
print('P-value:',pvalue)

P-value: 0.030446502691934697


<details>
  <summary><h4><strong>HINT 1</strong></h4></summary>

  Consider referencing the material on how to perform a two-sample t-test.
</details>

<details>
  <summary><h4><strong>HINT 2</strong></h4></summary>

  In `ttest_ind()`, a is the aqi column from the "New York" dataframe, an b is the aqi column from the "Ohio" dataframe.
</details>

<details>
  <summary><h4><strong>HINT 3</strong></h4></summary>

  You can assign `tstat`, `pvalue` to the output of `ttest_ind`. Be sure to include `alternative = less` as part of your code.  

</details>

#### **Question 3. What is your P-value for hypothesis 2, and what does this indicate for your null hypothesis?**

The P-value of 3% is less than our significance level of 5%, which implies we reject the null hypothesis and accept the alternative hypothesis that the mean AQI of NY is less than the mean AQI of Ohio.

###  Hypothesis 3: A new policy will affect those states with a mean AQI of 10 or greater. Can you rule out Michigan from being affected by this new policy?

Before proceeding with your analysis, it will be helpful to subset the data for your comparison.

In [103]:
# Create dataframes for each sample being compared in your test

### YOUR CODE HERE ###
# Create dataframe with states of mean aqi less than 10
states_mask = aqi.groupby(['state_name']).aqi.mean() >= 10
states = aqi.state_name.sort_values(ascending=True).unique()
states = states[states_mask]

Omask = aqi.state_name.isin(states)
states10_df = aqi[Omask]

# Create dataframe with Michigan data
Michiga_df = aqi[aqi.state_name=="Michigan"]

,Unnamed: 0,date_local,state_name,county_name,city_name,local_site_name,parameter_name,units_of_measure,arithmetic_mean,aqi
65,65,2018-01-01,Michigan,Wayne,Livonia,LIVONIA-NR,Carbon monoxide,Parts per million,0.338889,5
122,122,2018-01-01,Michigan,Wayne,Detroit,West corner,Carbon monoxide,Parts per million,0.394737,8
123,123,2018-01-01,Michigan,Wayne,Detroit,MARK TWAIN MIDDLE SCHOOL,Carbon monoxide,Parts per million,0.515789,9
129,129,2018-01-01,Michigan,Wayne,Detroit,ELIZA-NR,Carbon monoxide,Parts per million,0.616667,11
192,192,2018-01-01,Michigan,Wayne,Allen Park,Allen Park,Carbon monoxide,Parts per million,0.811111,13
207,207,2018-01-01,Michigan,Wayne,Not in a city,Eliza Downwind,Carbon monoxide,Parts per million,0.516667,10
226,226,2018-01-01,Michigan,Kent,Grand Rapids,GR-MONROE,Carbon monoxide,Parts per million,0.200000,2
242,242,2018-01-01,Michigan,Wayne,Detroit,(Northeast corner),Carbon monoxide,Parts per million,0.378947,7
248,248,2018-01-01,Michigan,Wayne,Detroit,NORTHWEST,Carbon monoxide,Parts per million,0.415789,8


<details>
  <summary><h4><strong>HINT 1</strong></h4></summary>

  Consider referencing the material on subsetting dataframes.  
</details>

<details>
  <summary><h4><strong>HINT 2</strong></h4></summary>

  Consider creating one dataframe which only includes Michigan.
</details>

#### Formulate your hypothesis:

**Formulate your null and alternative hypotheses here:**

*   $H_0$: The mean AQI of Michigan is less than or equal to 10.
*   $H_A$: The mean AQI of Michigan is greater than 10.


#### Significance Level (remains at 5%)

#### Determine the appropriate test procedure:

Here, you are comparing one sample mean relative to a particular value in one direction. Therefore, you will utilize a **one-sample  𝑡-test**. 

#### Compute the P-value

In [105]:
# Computer your p-value here

### YOUR CODE HERE ###
tstat, pval = stats.ttest_1samp(a=Michiga_df.aqi,popmean=10,alternative='greater')
print("P-value:",pval)

P-value: 0.9399405193140109


<details>
  <summary><h4><strong>HINT 1</strong></h4></summary>

  Consider referencing the material on how to perform a one-sample t-test.
</details>

<details>
  <summary><h4><strong>HINT 2</strong></h4></summary>

  In `ttest_1samp)`, you are comparing the aqi column from your Michigan data relative to 10, the new policy threshold.
</details>

<details>
  <summary><h4><strong>HINT 3</strong></h4></summary>

  You can assign `tstat`, `pvalue` to the output of `ttest_1samp`. Be sure to include `alternative = greater` as part of your code.  

</details>

#### **Question 4. What is your P-value for hypothesis 3, and what does this indicate for your null hypothesis?**

With a P-value of 93% we have overwheliming evidence against the null hypothesis. This leads us to conclude that Michigan as a mean AQI value of less than 10.

## Step 4. Results and Evaluation

Now that you've completed your statistical tests, you can consider your hypotheses and the results you gathered.

#### **Question 5. Did your results show that the AQI in Los Angeles County was statistically different from the rest of California?**

There is no statistical significance between the difference of the mean AQI of California and LA

#### **Question 6. Did New York or Ohio have a lower AQI?**

The mean AQI of NY is less than the mean AQI of Ohio.

#### **Question 7: Will Michigan be affected by the new policy impacting states with a mean AQI of 10 or greater?**



No, Michigan does not show statistical evidence of having a mean AQI greater than 10.

# Conclusion

**What are key takeaways from this lab?**

**What would you consider presenting to your manager as part of your findings?**

**What would you convey to external stakeholders?**


**Congratulations!** You've completed this lab. However, you may not notice a green check mark next to this item on Coursera's platform. Please continue your progress regardless of the check mark. Just click on the "save" icon at the top of this notebook to ensure your work has been logged.